In [2]:
import os 
import json 
import glob
from tqdm import tqdm 
import re
import ast

In [5]:
test_out_store_path = "/data3/tydata3/code_optimization/cpp/cpp_binary/test_out/"
cpp_out_files = glob.glob(test_out_store_path + "*.out")
print(f"There are {len(cpp_out_files)} in {test_out_store_path}")

There are 9694 in /data3/tydata3/code_optimization/cpp/cpp_binary/test_out/


In [6]:
test_benchmark_store_path = "/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out"
problems_path = os.listdir(test_benchmark_store_path)
print(f"There are {len(problems_path)} problems in test out.")

There are 123 problems in test out.


In [15]:
def is_answer_correct(input_case_path, stdout_bin):
    """
    check stdout_bin is correct 
    """
    output_case_path = input_case_path.replace("input", "output")
    with open(output_case_path, 'r') as g:
        truth = g.read().strip()
    
    ground_truth_lines = truth.strip().splitlines()
    output_lines = stdout_bin.strip().splitlines()

    IsCorrect = True
    for gen_output, ground_truth_output in zip(output_lines, ground_truth_lines):
        is_corr = gen_output == ground_truth_output
        if not is_corr:
            try:
                gen_output = float(gen_output)
                ground_truth_output = float(ground_truth_output)
                is_corr = abs(gen_output - ground_truth_output) < 1e-3
            except:
                pass
        
        if not is_corr:
            IsCorrect = False
    
    return IsCorrect

In [24]:
def benchmark_postprocess(test_binary_benchmark_dir):
    # for each test binary benchmark dir
    benchmark_results_path = "testcases_all_benchmark_results.json"
    with open(os.path.join(test_binary_benchmark_dir, benchmark_results_path), 'r') as f:
        results = json.load(f)
    
    final_result = {
        "binary_exec_right_and_gem5_right":{"testcases_id": [],"binary_exec_right_and_gem5_right_and_answer_correct":[], "binary_exec_right_and_gem5_right_and_answer_wrong":[]},
        "binary_exec_right_and_gem5_wrong":[],
        "binary_exec_wrong_and_gem5_right":[],
        "binary_exec_wrong_and_gem5_wrong":[],
        "testcases_number": 0
    }

    for testcase_result in results:
        returncode_bin = testcase_result["returncode_bin"]
        returncode_gem5 = testcase_result["returncode_gem5"]
        test_case_id = int(testcase_result["test_case_id"])
        
        if returncode_bin == 0 and returncode_gem5 == 0:
            final_result["binary_exec_right_and_gem5_right"]["testcases_id"].append(test_case_id)
            Is_answer_correct = is_answer_correct(testcase_result["input_case_path"], testcase_result["stdout_bin"])
            if Is_answer_correct:
                final_result["binary_exec_right_and_gem5_right"]["binary_exec_right_and_gem5_right_and_answer_correct"].append(test_case_id)
            else:
                final_result["binary_exec_right_and_gem5_right"]["binary_exec_right_and_gem5_right_and_answer_wrong"].append(test_case_id)
        elif returncode_bin == 0 and returncode_gem5 != 0:
            final_result["binary_exec_right_and_gem5_wrong"].append(test_case_id)
        elif returncode_bin != 0 and returncode_gem5 == 0:
            final_result["binary_exec_wrong_and_gem5_right"].append(test_case_id)
        else:
            final_result["binary_exec_wrong_and_gem5_wrong"].append(test_case_id)
        
        final_result["testcases_number"] += 1

    with open(os.path.join(test_binary_benchmark_dir, "analysis_result.json"), 'w') as gg:
        json.dump(final_result, gg, indent=4)

    return None

In [33]:
test_benchmark_store_path = "/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out"
problems = os.listdir(test_benchmark_store_path)
# test_binary_benchmark_dir = os.path.join(test_benchmark_store_path, "p01717", "u923320778", "s651875974")
# benchmark_postprocess(test_binary_benchmark_dir)

for problem in tqdm(problems):
    problem_path = os.path.join(test_benchmark_store_path, problem)
    users = os.listdir(problem_path)
    for user in users:
        user_path = os.path.join(test_benchmark_store_path, problem, user)
        submissions = os.listdir(user_path)
        for submission in submissions:
            submission_path = os.path.join(test_benchmark_store_path, problem, user, submission)
            test_binary_benchmark_dir = submission_path
            benchmark_postprocess(test_binary_benchmark_dir)

100%|██████████| 123/123 [01:46<00:00,  1.15it/s]


In [36]:
def result_statistics(analysis_result):
    with open(analysis_result, 'r') as f:
        data = json.load(f)
    
    count = len(data["binary_exec_right_and_gem5_right"]["binary_exec_right_and_gem5_right_and_answer_correct"])
    if count == data["testcases_number"]:
        return True
    else:
        return False

In [40]:
count = 0
count_all_right = 0
all_right_binary = []
for problem in tqdm(problems):
    problem_path = os.path.join(test_benchmark_store_path, problem)
    users = os.listdir(problem_path)
    for user in users:
        user_path = os.path.join(test_benchmark_store_path, problem, user)
        submissions = os.listdir(user_path)
        for submission in submissions:
            count += 1
            submission_path = os.path.join(test_benchmark_store_path, problem, user, submission)
            result = result_statistics(os.path.join(submission_path, "analysis_result.json"))
            if result:
                count_all_right += 1
                binary = os.path.join("/data3/tydata3/code_optimization/cpp/cpp_binary/test_out", f"{problem}_{submission}_{user}.out")
                all_right_binary.append(binary)

print(f"count = {count}")
print(f"all right count = {count_all_right}")
with open(os.path.join("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out_all_right.txt"), 'w') as f:
    for item in all_right_binary:
        f.write(item + '\n')

100%|██████████| 123/123 [00:00<00:00, 333.43it/s]

count = 9694
all right count = 5840


In [3]:
def calculate_sim_seconds(stats):
    # more accurate than sim_seconds
    return float(stats["sim_ticks"]) / float(stats["sim_freq"])

In [5]:
def parse_stats_txt(gem5_stats_path):
    with open(gem5_stats_path, 'r') as f:
        stats_lines = f.readlines()
    
    stats = {}
    for line in stats_lines:
        if line.strip() == '':
            continue 
        if "Begin" in line:
            continue
        if "End" in line:
            continue
        line = re.sub("#.*", "", line).strip() # remove comments
        parts = line.split()
        parts = [part.strip() for part in parts]
        if len(parts) > 2:
            value = parts[1:]
        elif len(parts) == 2:
            value = parts[1]
        else:
            print(f"could not parse line {line}")
            continue
        key = parts[0]
        if isinstance(value, str):
            try:
                value = value.replace("%", "").replace("nan", "None").replace("inf", "None").replace("-inf", "None")
                value = ast.literal_eval(value) if value != "None" else None
            except:
                print(f"could not parse value {value} for key {key}")
        elif isinstance(value, list):
            try:
                value = [v.replace("%", "").replace("nan", "None").replace("inf", "None").replace("-inf", "None") for v in value]
                value = [ast.literal_eval(v) if v!= "None" else None for v in value]
            except:
                print(f"could not parse value {value} for key {key}")
        
        stats[key] = value
    stats["sim_seconds_precise"] = calculate_sim_seconds(stats)
    return stats

In [6]:
def get_average_time(binary):
    bin_file_path = os.path.join(binary)

    pattern_problem_id = r"out/(p\d+)"
    pattern_submission_id = r's\d+'
    pattern_user_id = r'u\d+'

    match_problem_id = re.findall(pattern_problem_id, bin_file_path)[0]
    match_submission_id = re.findall(pattern_submission_id, bin_file_path)[0]
    match_user_id = re.findall(pattern_user_id, bin_file_path)[0]

    base_path = "/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out"
    testcases_path = os.path.join(base_path, f"{match_problem_id}", f"{match_user_id}", f"{match_submission_id}/")
    
    sim_seconds_precise_all = []
    gem5_stats = glob.glob(testcases_path + "gem5_stats.*.txt")
    for gem5_stat in gem5_stats:
        stats = parse_stats_txt(gem5_stat)
        sim_seconds_precise = stats["sim_seconds_precise"]
        sim_seconds_precise_all.append(sim_seconds_precise)

    return sum(sim_seconds_precise_all) / len(sim_seconds_precise_all)

In [7]:
with open(os.path.join("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out_all_right.txt"), 'r') as f:
    binarys = f.read().splitlines()

results = {}
for binary in tqdm(binarys):
    average_sim_seconds_precise = get_average_time(binary)

    pattern_problem_id = r"out/(p\d+)"
    pattern_submission_id = r's\d+'
    pattern_user_id = r'u\d+'

    match_problem_id = re.findall(pattern_problem_id, binary)[0]
    match_submission_id = re.findall(pattern_submission_id, binary)[0]
    match_user_id = re.findall(pattern_user_id, binary)[0]

    if match_problem_id in results:
        if match_user_id in results[match_problem_id]:
            if match_submission_id in results[match_problem_id][match_user_id]:
                print("Error.")
            else:
                results[match_problem_id][match_user_id][match_submission_id] = average_sim_seconds_precise
        else:
            results[match_problem_id][match_user_id] = {}
            results[match_problem_id][match_user_id][match_submission_id] = average_sim_seconds_precise
    else:
        results[match_problem_id] = {}
        results[match_problem_id][match_user_id] = {}
        results[match_problem_id][match_user_id][match_submission_id] = average_sim_seconds_precise


with open(os.path.join("/data3/tydata3/code_optimization/cpp/benchmark_gem5_testcases_all/test_out_times.json"), 'w') as f:
    json.dump(results, f, indent=4)

  0%|          | 0/5840 [00:00<?, ?it/s]

100%|██████████| 5840/5840 [3:46:15<00:00,  2.32s/it]  
